In [1]:
from df_functions import read_csv_file, extract_users, split_user_df
from calculations import longest_consec, calc_avg_timediff, get_variance_score, create_vectorizer
from sessionID_check import avg_tokens_5mins
import pandas as pd


In [4]:
print('test push')

test push


Hur kommer ett request se ut när det kommer in?
- sträng i Redis? 
- sparas i sorted sets?
- hämta som mest ett fixt antal, ex 50.

Efter allt är hämtat från Redis:
- spara undan i en data frame med varje värde i rätt kolumn.

Dataframe:

userID, sessionID, URL, osv.        time mellan req         unika sessionID
user1   session1, login
user1   session1  chat
user1   session1  sendMessage

Kör in data framen i en funktion som gör nedanstående steg med hjälp av diverse helper-metoder:
För user1: 
- vad är den genomsnittliga tiden mellan requests?
- tokens under 5 min (skapa en funktion som först grupperar och sen räknar ut medelvärde av antal sessionIDs för alla 5-min intervall)
När dessa två steg är klara, behöver vi inte längre ta hänsyn till timestamps eller sessionID. Resultaten från dessa beräkningar 
sparas undan i egna variabler som läggs till i slutskedet av konstruktionen av vår test-dataframe.

- vektorisera URLer och ta fram:
    - varians, 
    - unika requests,
    - längsta följd av upprepade förfrågningar

In [2]:
"""Code for fetching the data"""

#Fetch data from csv file
data = read_csv_file('requests.csv')

#Extracts a list of all unique users in dataframe
users = extract_users(data)
vect = create_vectorizer(data)

#Fetches the first user in list
#user1 = users[6]

#data_chunk = split_user_df(data, user1)[0]


#longest_consec(data_chunk)


In [3]:
"""
Code for creating csv-file containing information about average request frequency, average number of tokens per 5 mins, 
longest consecutive chain of repetitive requests for every chunk of 50 requests for each user
"""

df_calculations = pd.DataFrame()
for user in users:
    data_chunks = split_user_df(data, user)
    avg_req_frequences = []
    avg_tokens = []
    repetitions = []
    var_scores = []

    for i in range(0, len(data_chunks)):
        avg_req_frequences.append(calc_avg_timediff(data_chunks[i]))
        avg_tokens.append(avg_tokens_5mins(data_chunks[i]))
        repetitions.append(longest_consec(data_chunks[i]))
        var_scores.append(get_variance_score(data_chunks[i], vect))
    
    user_df = pd.DataFrame()
    user_df['request_freq'] = avg_req_frequences
    user_df['avg_tokens'] = avg_tokens
    user_df['longest_consec'] = repetitions
    user_df['var_score'] = var_scores
    user_df['user'] = user
    df_calculations = pd.concat([df_calculations, user_df])

df_calculations.reset_index(inplace=True)
df_calculations = df_calculations.drop(columns='index')
df_calculations.to_csv('calculations.csv', index=False) 

KeyboardInterrupt: 

In [ ]:
data_calc = pd.read_csv('calculations.csv')
data_calc

,request_freq,avg_tokens,longest_consec,var_score,user
0,2.036531,2.000000,15,172.095238,user-1003
1,2.572653,4.000000,9,140.971429,user-1003
2,2.268571,2.000000,15,149.238095,user-1003
3,2.132122,2.000000,10,149.238095,user-1003
4,2.198367,3.000000,13,150.685714,user-1003
...,...,...,...,...,...
5749,642.863652,0.111111,5,24.352381,user-754
5750,1069.710038,0.118280,3,8.838095,user-641
5751,756.243600,0.153846,2,8.266667,user-641
5752,1295.875074,0.076271,3,8.352381,user-715
